# Experta: Theory Fundamentals of Production Systems and RETE Algorithm

---

## Introduction

Experta is a Python library based on Production Systems, also known as Expert Systems or Rule-Based Systems, wich is a fundamental paradigm in simbolic AI.

---

## What are Expert Systems? 

An expert system is formed by 3 key components:

1. **Facts base**: it stores the factual knowledge of the system. That is, what the system knows in a specific moment.

2. **Rule Base**: contains the procedure knowledge as "if-then" rules.

3. **Inference motor**: motor that determines whether apply one rule or another and when to apply it.

The central idea is no model the thinking as a rule chaining process, simillar at how human experts would take a decission applying their knowledge to a specific subject.


---

## RETE Algorithm

Experta´s core is RETE algorithm, designed in 1982 by Charles L.Forgy. This algorithm is crucal for knowing what Experta is doing and also knowing why it is efficient:

- Its **function** is to optimize the coincidence of patterns between facts and rules. 

- It builds a node network that represent patterns. Then, it avoids reevaluating all the rules when the facts change.

Therefore, RETE builds a "discrimination net" that acts as an efficient filter for determining which rules should be activated in response to changes made on the facts base.

---

## Experta´s execution cycle

1. **Match**: the motor identifies all the rules that can be activaded with the actual facts base.
2. **Conflict Resolution**: if multiple rules coincide, the motor decides which one is executed first (using conflic resolution strategies).
3. **Act**: it executes the action associated with the selected rule, that tipically modifies the facts base.
4. **Cycle**: the process is repeated until there are no more rules to be activated.

This cycle is knwon as "cycle recognize-act" or "production cycle".

---

## Declarative vs Imperative Programming

Experta is a **Declarative Programming paradigm**, in contrast with traditional programming. Instead of defining HOW to make something step by step, Declarative Programming specifies WHICH conditions need to be acomplished. In Experta, the developer defines rules declaratively and the motor is the one that says when and how are they going to be applied. 

---

## Relevance for F1 Strategy

Experta´s selection for the F1 strategy problem is theoretically justified due to this 5 points:

1. **Codificable expert knowledge**: F1 strategies can be expressed naturally as conditional rules based on expert knowledge.
2. **Incremental Reasoning**: during the race, information comes continously, such as radios, telemetry, track data or weather. RETE is efficient for updating conclussions based on new information.
3. **Knowledge Explanation**: rules can be read and modified by humans, allowing adjusting strategies based on feedback.
4. **Explainable Capacity**: unlike black box models like Neural Networks, a system based on rules can explain exactly which conditions made them make a decission.
5. **Multiple Information Integration**: key for my project, as it brings me the capacity to merge structured information as data with semi-structured information like NLP radio analysis or my prediction models in the same logical framework.

The implementation through Experta allows to capture strategic reasoning of F1 Teams, creating a system that emulates how they would take real-time decissions based on the actual avaliable information.

---

## Basic Example of Decision Making with an Expert System

I will illustrate a simple case: deciding whether an F1 car should pit based on tire degradation and weather conditions.

### 1. Problem Definition
We have basic rules to decide on a pit stop:

- If tire degradation is greater than 60%, recommend a pit stop.
- If it is raining and the car has dry tires, recommend a pit stop.
- If the degradation is moderate (30-60%) and the driver reports grip issues, recommend a pit stop.

### 2. Step-by-Step Process
Here is what happens when we run Experta:

#### 2.1 Initialization:
- The rule engine `EstrategiaF1` is created.
- The method `reset()` is called to prepare the engine.

#### 2.2 Declaration of Facts:
- We add that the tires have a degradation of 45% and are of the dry type.
- We add that it is not raining.
- We add that the driver reports grip issues.

#### 2.3 Execution of the RETE Cycle:
- The engine calls `run()`, starting the inference cycle.
- RETE builds an activation network with the three defined rules.

#### 2.4 Rule Evaluation:
- **First rule (very_degraded_tires):** DOES NOT MATCH because degradation = 45% (less than 60%).
- **Second rule (change_to_rain):** DOES NOT MATCH because raining = False.
- **Third rule (moderate_grip_problems):** MATCHES because:
  - degradation = 45% (is between 30% and 60%)
  - the message contains "problems" and "grip"

#### Rule Activation:
- The rule `moderate_grip_problems` is activated.
- Its action is executed, declaring a new fact: **Recommendation**.

#### New Evaluation Cycle:
- The engine evaluates whether there are new rules that match the newly added fact.
- In this case, no additional rules are activated.
- The engine terminates the execution.

### Final Result:
We obtain a recommendation:
- **Action recommended:** pit
- **Reason:** Grip issues reported with moderate degradation
- **Urgency:** medium

This is the essence of how the expert system processes the rules: it continuously evaluates the available facts against the rule conditions and executes the corresponding actions when matches occur. The RETE algorithm makes this process efficient by avoiding the need to re-evaluate all rules for every fact.


---

## 1. Import the Libraries

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

---

## 2. Definition of Fact Classes

---

In [1]:
from experta import Fact, Field

AttributeError: module 'collections' has no attribute 'Mapping'

Field object takes 5 possible arguments:

1. Datatype(mandatory) specifies the expected data type. 
2. Default(optional) specifies a default value if none is given.
3. Mandatory(optional) is a boolean to put if the Field is mandatory.
4. Optional, contrary to Mandaroty.
5. Test (function) allows defininf a function to validate the value.

### 2.1 Telemetry Facts

In [ ]:
# Telemetry Facts
class TelemetryFact(Fact):
    """
    Facts about car telemetry and performance
    """
    lap_time = Field(float, mandatory= False)           # Curremt lap time
    predicted_lap_time = Field(float, mandatory=False)  # Predicted lap time by the model
    tire_age = Field(int, mandatory=False)              # Age of the current tire set in laps
    compound_id = Field(int, mandatory=False)           # Tire type with ID
    position = Field(int, mandatory= False)             # Current race position
    driver_number = Field(int, mandatory=False)         # Driver number 


### 2.2 Degradation Facts

In [ ]:
class DegradationFact(Fact):
    """
    Facts about tire degradation
    """
    degradation_rate = Field(float, mandatory= False)           # Seconds lost per lap due to degradation
    previous_rates = Field(list, mandatory= False)              # Last N degradation rates for trend analysis
    fuel_adjusted_deg_percent = Field(float, mandatory= False)  # Percentage degradation adjusted for fuel 

### 2.3 Gap Facts

In [ ]:
class GapFact(Fact):
    """
    Facts about gaps to other cars
    """
    gap_ahead = Field(float, mandatory= False)          # Time to car ahead (seconds)
    gap_behind = Field(float, mandatory= False)         # Time to car behind (seconds)
    gap_ahead_trend = Field(float, mandatory= False)    # Change in gap ahead over last laps
    gap_behind_trend = Field(float, mandatory= False)   # Change in gap behind over last laps

### 2.4 Radio Facts

In [ ]:
class RadioFact(Fact):
    """
    Facts from radio communications analysis
    """
    sentiment = Field(str, mandatory= False)  # positive, negative, neutral
    intent = Field(str, mandatory= False)     # WARNING, QUESTION, etc.
    entities = Field(dict, mandatory= False)  # Detected entities categorized (SITUATION, INCIDENT, PIT_CALL, etc)
    timestamp = Field(float, mandatory= False)# When the message was received

### 2.5 Race Status

In [ ]:
class RaceStatusFact(Fact):
    """
    Facts about current race status
    """
    lap = Field(int, mandatory= True)               # Current lap
    total_laps = Field(int, mandatory= True)        # Total race laps
    race_phase = Field(str, mandatory= False)       # start, mid, end
    track_status = Field(str, mandatory= False)     # clear, yellow, safety car, red flag
    

### 2.6 Strategy Reccomendation

In [ ]:
class StrategyRecommendation(Fact):
    """
    Reccommendation produced by the Expert System
    """
    action = Field(str, mandatory= True)                        # Specific action to take
    confidence = Field(float, mandatory= True)                  # Confidende level (0-1)
    explanation = Field(str, mandatory= True)                   # Natural Language Explanation
    priority = Field(int, mandatory= False, default = 0)        # Priority level (higher = more urgent)
    lap_issued = Field(int, mandatory= True)                    # Lap when reccomendation was made

---

## 3. Engine Definition with Rule Documentation